In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Research Project

/content/drive/My Drive/Research Project


In [ ]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pip install -U mittens

In [ ]:
from numpy import asarray
from numpy import zeros

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold

from statistics import mean, stdev, median, mode

import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

from mittens import GloVe, Mittens
from hyperopt import fmin, tpe, hp

In [ ]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import MaxPool3D, AveragePooling3D

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [ ]:
# pip install autocorrect

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Dropout, MaxPool1D
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Bidirectional
import tensorflow.keras.backend as K

### Getting Data

In [ ]:
# data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 583 Data Mining and Text Mining\Research Project\Data'

In [ ]:
data_path = r'/content/drive/My Drive/Research Project/Data'

In [ ]:
romney_train_pr_df = pd.read_csv(os.path.join(data_path, 'Romney Training Data.csv'), usecols = [1,2]).dropna()
romney_val_pr_df = pd.read_csv(os.path.join(data_path, 'Romney Validation Data.csv'), usecols = [1,2]).dropna()

In [ ]:
romney_train_pr_df.head()

,Doc Text,Sentiment
0,believe romney able work w bipartisim gov brin...,Positive
1,ask romney past decade tax end obama debate pr...,Positive
2,conclusion romney get own,Positive
3,register democrat nothing give reason change v...,Positive
4,aunt buy theiphone tell need vote romne shecray,Neutral


In [ ]:
romney_val_pr_df.head()

,Doc Text,Sentiment
0,vote romney waystogetshot,Neutral
1,waystogetshot voting romney,Negative
2,romney go stand china afraid go view,Neutral
3,swing state poll woman push romney lead via,Neutral
4,least job come back romney nba team outsource ...,Negative


In [ ]:
romney_train1_pr_df = pd.read_csv(os.path.join(data_path, 'Romney Training1 Data.csv'), usecols = [1,2]).dropna()

In [ ]:
romney_train1_pr_df.head()

,Doc Text,Sentiment
0,insidious mitt romney bain help philip morris ...,Negative
1,mean like romney cheat primary,Negative
2,mitt romney still believe black president,Negative
3,romney tax plan deserve nd look secret one dif...,Negative
4,hope romney debate prepped people last time,Positive



## Building CNN Models

In [ ]:


# int_sequences_input = layers.Input(shape=(None,), dtype="int64")
# embedded_sequences = embedding_layer(int_sequences_input)
# x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
# x = layers.MaxPooling1D(5)(x)
# x = layers.Conv1D(128, 5, activation="relu")(x)
# x = layers.MaxPooling1D(5)(x)
# x = layers.Conv1D(128, 5, activation="relu")(x)
# x = layers.GlobalMaxPooling1D()(x)
# x = layers.Dense(128, activation="relu")(x)
# x = layers.Dropout(0.5)(x)
# preds = layers.Dense(len(class_names), activation="softmax")(x)
# model = keras.Model(int_sequences_input, preds)
# model.summary()


In [ ]:
def one_hot(data):
    data = np.asarray(data)
    temp = np.zeros((len(data),3))
#     print(data[0])
    for i in range(len(temp)):
        if data[i] == 'Negative':
            temp[i][2] = 1 ## Negative sentiment third neuron
        elif data[i] == 'Neutral':
            temp[i][1] = 1 ## Neutral sentiment second neuron  
        else:
            temp[i][0] = 1 ## Positive sentiment first neuron 

    return temp
    
    


In [ ]:
def pred(x):
    temp = []
    for i in x:
        m = np.argmax(i)
        if m == 0:
            temp.append('Positive')
        elif m == 1:
            temp.append('Neutral')
        else:
            temp.append('Negative')
    return temp

## Building Glove Dictionary

In [ ]:
embeddings = {}
with open(os.path.join(data_path,"glove.twitter.27B.200d.txt"), 'r', encoding="utf-8") as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = asarray(values[1:], dtype='float32')
        embeddings[word] = vector

## Embedding Matrix Function

In [ ]:
def emb_matrix(t,embeddings, we_dim):
    # creating a embedding matrix for the words in training data, which will be used as weight matrix for embedding layer
    vocab_size = len(t.word_index) + 1    
    embedding_matrix = zeros((vocab_size, we_dim))
    for word, i in t.word_index.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix, vocab_size

## Fine tuning the word embeddings of 300 dimensions using mittens library



## Used the code for finetuning from the following link:
### https://towardsdatascience.com/fine-tune-glove-embeddings-using-mittens-89b5f3fe4c39

In [ ]:
def finetune(training): 
    training_tokens = [word_tokenize(i) for i in training['Doc Text']]
    #training_tokens

    oov = [j for i in training_tokens for j in i if j not in embeddings.keys()]
    print(len(oov))

    corp_vocab = list(set(oov))

    cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
    trr =''
    for i in training_tokens:
        for j in i:
            trr+= j
            trr += ' '

    # print(trr)
    # print(z)
    X = cv.fit_transform([trr])
    Xc = (X.T * X)
    Xc.setdiag(0)
    coocc_ar = Xc.toarray()

    mittens_model = Mittens(n=200, max_iter=len(oov)+200)

    new_embeddings = mittens_model.fit(
      coocc_ar,
      vocab=corp_vocab,
      initial_embedding_dict= embeddings)

    new_embeddings = dict(zip(corp_vocab, new_embeddings))
    return training_tokens, new_embeddings
  


In [ ]:
embeddings2= embeddings.copy()

training_tokens, new_embeddings = finetune(romney_train_pr_df)
embeddings2.update(new_embeddings)

573
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Iteration 770: loss: 0.011159378103911877

In [ ]:
oov2 = [j for i in training_tokens for j in i if j not in embeddings2.keys()]
print(len(oov2))

0


## Using CNN

In [ ]:
def cross_valid_cnn(X,y,epochs,batch_size,max_length, learning_rate, we_dim):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)

        X_train1, X_val, y_train1, y_val = X.iloc[train_index], X.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
        tokenise_tf = Tokenizer()
        tokenise_tf.fit_on_texts(X_train1) 
    
        encoded_train = tokenise_tf.texts_to_sequences(X_train1)
        training_padded = pad_sequences(encoded_train, maxlen=max_length, padding='post', truncating = 'pre')
        embedding_matrix, vocab_size = emb_matrix(tokenise_tf, embeddings, we_dim)

        encoded_validation = tokenise_tf.texts_to_sequences(X_val)
        validation_padded = pad_sequences(encoded_validation, maxlen=max_length, padding='post', truncating = 'pre')

        adam_optimizer = optimizers.Adam(learning_rate=learning_rate)

        embedding_layer = Embedding(vocab_size, we_dim, embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False)
        int_sequences_input = layers.Input(shape=(None,), dtype="int64")
        embedded_sequences = embedding_layer(int_sequences_input)
        x = layers.Conv1D(128, 3)(embedded_sequences)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = layers.MaxPooling1D(2)(x)
        x = layers.Conv1D(128, 3)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = layers.MaxPooling1D(2)(x)

        x = layers.Conv1D(128, 3)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = layers.GlobalMaxPooling1D()(x)
        # x = layers.MaxPooling1D(2)(x)

        x = layers.Flatten()(x)
        x = layers.Dense(128, activation="relu")(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(128, activation="relu")(x)
        x = layers.Dropout(0.2)(x)
        preds = layers.Dense(3, activation="softmax")(x)

        model = tf.keras.Model(int_sequences_input, preds)
        # print(model.summary())
        # print(z)
        model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['acc'])
        history = model.fit(training_padded, one_hot(y_train1), epochs=epochs, verbose=0, batch_size=batch_size, shuffle =True)
        y_pred_temp = model.predict(validation_padded)
        y_pred = pred(y_pred_temp)
        f1_temp = f1_score(y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[2])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[0])

        acc.append(round(accuracy_score(y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

In [ ]:
def model_cnn1(embedding_matrix, vocab_size, epochs,batch_size,max_length, learning_rate, we_dim):

    adam_optimizer = optimizers.Adam(learning_rate=learning_rate)

    embedding_layer = Embedding(vocab_size, we_dim, embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False)
    int_sequences_input = layers.Input(shape=(None,), dtype="int64")
    embedded_sequences = embedding_layer(int_sequences_input)

    x = layers.Conv1D(64, 3)(embedded_sequences)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(128, 3)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = layers.MaxPooling1D(2)(x)

    x = layers.Conv1D(256, 3)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = layers.GlobalMaxPooling1D()(x)
    # x = layers.MaxPooling1D(2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="tanh")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation="tanh")(x)
    x = layers.Dropout(0.2)(x)
    preds = layers.Dense(3, activation="softmax")(x)

    model = tf.keras.Model(int_sequences_input, preds)
    # print(model.summary())
    # print(z)
    model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['acc'])
    return model


In [ ]:
def model_cnn2(embedding_matrix, vocab_size, epochs,batch_size,max_length, learning_rate, we_dim):

    adam_optimizer = optimizers.Adam(learning_rate=learning_rate)

    embedding_layer = Embedding(vocab_size, we_dim, embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False)
    int_sequences_input = layers.Input(shape=(None,), dtype="int64")
    embedded_sequences = embedding_layer(int_sequences_input)

    x = layers.Conv1D(32, 3)(embedded_sequences)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(64, 3)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = layers.GlobalMaxPooling1D()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="tanh")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation="tanh")(x)
    x = layers.Dropout(0.2)(x)
    preds = layers.Dense(3, activation="softmax")(x)

    model = tf.keras.Model(int_sequences_input, preds)
    # print(model.summary())
    # print(z)
    model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['acc'])
    return model


In [ ]:
def model_cnn3(embedding_matrix, vocab_size, epochs,batch_size,max_length, learning_rate, we_dim):

    adam_optimizer = optimizers.Adam(learning_rate=learning_rate)

    embedding_layer = Embedding(vocab_size, we_dim, embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False)
    int_sequences_input = layers.Input(shape=(None,), dtype="int64")
    embedded_sequences = embedding_layer(int_sequences_input)

    x = layers.Conv1D(64, 3)(embedded_sequences)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = layers.GlobalMaxPooling1D()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="tanh")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation="tanh")(x)
    x = layers.Dropout(0.2)(x)
    preds = layers.Dense(3, activation="softmax")(x)

    model = tf.keras.Model(int_sequences_input, preds)
    # print(model.summary())
    # print(z)
    model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['acc'])
    return model


In [ ]:
# max_length = 31
# epochs = 2
# batch_size = 64
# learning_rate = 0.001
# we_dim = 200
# par_dict = {'max_length': max_length, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs}
# print(par_dict)

# tokenise_tf = Tokenizer()
# tokenise_tf.fit_on_texts(romney_train_pr_df['Doc Text']) 

# encoded_train = tokenise_tf.texts_to_sequences(romney_train_pr_df['Doc Text'])
# training_padded = pad_sequences(encoded_train, maxlen=max_length, padding='post', truncating = 'pre')
# embedding_matrix, vocab_size = emb_matrix(tokenise_tf, embeddings, we_dim)

# encoded_validation = tokenise_tf.texts_to_sequences(romney_val_pr_df['Doc Text'])
# validation_padded = pad_sequences(encoded_validation, maxlen=max_length, padding='post', truncating = 'pre')

# try: 
#   model = model_cnn1(embedding_matrix, vocab_size, epochs,batch_size,max_length, learning_rate, we_dim)
#   history = model.fit(training_padded, one_hot(romney_train_pr_df['Sentiment']), epochs=epochs, verbose=0, batch_size=batch_size, shuffle =True)

# except:
#   try: 
#     model = model_cnn2(embedding_matrix, vocab_size, epochs,batch_size,max_length, learning_rate, we_dim)
#     history = model.fit(training_padded, one_hot(romney_train_pr_df['Sentiment']), epochs=epochs, verbose=0, batch_size=batch_size, shuffle =True)
#   except:

#     model = model_cnn3(embedding_matrix, vocab_size, epochs,batch_size,max_length, learning_rate, we_dim)
#     history = model.fit(training_padded, one_hot(romney_train_pr_df['Sentiment']), epochs=epochs, verbose=0, batch_size=batch_size, shuffle =True)


# y_pred_temp = model.predict(validation_padded)
# y_pred = pred(y_pred_temp)
# f1_temp = f1_score(romney_val_pr_df['Sentiment'], y_pred, average = None)

# f1_Positive = f1_temp[2]
# f1_Neutral = f1_temp[1]
# f1_Negative = f1_temp[0]

# accuracy = round(accuracy_score(romney_val_pr_df['Sentiment'], y_pred),3) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)   
# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# print('\n')
# print(eval_dict)

In [ ]:
def objective_func_CNN(args):
    max_length = args['max_length']
    batch_size = args['batch_size']
    learning_rate = args['learning_rate']
    epochs = args['epochs']

    par_dict = {'max_length': max_length, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs}
    print(par_dict)

    tokenise_tf = Tokenizer()
    tokenise_tf.fit_on_texts(romney_train_pr_df['Doc Text']) 

    encoded_train = tokenise_tf.texts_to_sequences(romney_train_pr_df['Doc Text'])
    training_padded = pad_sequences(encoded_train, maxlen=max_length, padding='post', truncating = 'pre')
    embedding_matrix, vocab_size = emb_matrix(tokenise_tf, embeddings, we_dim)

    encoded_validation = tokenise_tf.texts_to_sequences(romney_val_pr_df['Doc Text'])
    validation_padded = pad_sequences(encoded_validation, maxlen=max_length, padding='post', truncating = 'pre')

    try: 
        model = model_cnn1(embedding_matrix, vocab_size, epochs,batch_size,max_length, learning_rate, we_dim)
        history = model.fit(training_padded, one_hot(romney_train_pr_df['Sentiment']), epochs=epochs, verbose=0, batch_size=batch_size, shuffle =True)

    except:
        try:            
            model = model_cnn2(embedding_matrix, vocab_size, epochs,batch_size,max_length, learning_rate, we_dim)
            history = model.fit(training_padded, one_hot(romney_train_pr_df['Sentiment']), epochs=epochs, verbose=0, batch_size=batch_size, shuffle =True)
        except:

            model = model_cnn3(embedding_matrix, vocab_size, epochs,batch_size,max_length, learning_rate, we_dim)
            history = model.fit(training_padded, one_hot(romney_train_pr_df['Sentiment']), epochs=epochs, verbose=0, batch_size=batch_size, shuffle =True)


    y_pred_temp = model.predict(validation_padded)
    y_pred = pred(y_pred_temp)
    f1_temp = np.round(f1_score(romney_val_pr_df['Sentiment'], y_pred, average = None),3)

    f1_Positive = f1_temp[2]
    f1_Neutral = f1_temp[1]
    f1_Negative = f1_temp[0]

    accuracy = round(accuracy_score(romney_val_pr_df['Sentiment'], y_pred),3) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)   
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print('\n')
    print(eval_dict)

    return -(eval_score)

space = {'max_length': hp.choice('max_length',range(4,60)),  
        'batch_size': hp.choice('batch_size', [32, 64, 128]),
         'epochs': hp.choice('epochs',range(5,30)), 
         'learning_rate': hp.uniform('learning_rate', 0,0.01)
        }                  
                                    
we_dim = 200                               
best_CNN = fmin(objective_func_CNN, space, algo=tpe.suggest, max_evals=20)
print(best_CNN)

{'max_length': 15, 'batch_size': 32, 'learning_rate': 0.0027327740726435803, 'epochs': 16}
{'accuracy': 0.189, 'f1_pos': 0.318, 'f1_neu': 0.0, 'f1_neg': 0.0, 'eval_score': 0.292}
{'max_length': 59, 'batch_size': 128, 'learning_rate': 0.005185465135720898, 'epochs': 25}
{'accuracy': 0.504, 'f1_pos': 0.444, 'f1_neu': 0.44, 'f1_neg': 0.584, 'eval_score': 0.456}
{'max_length': 59, 'batch_size': 64, 'learning_rate': 0.007631126012370996, 'epochs': 24}
{'accuracy': 0.539, 'f1_pos': 0.442, 'f1_neu': 0.403, 'f1_neg': 0.658, 'eval_score': 0.461}
{'max_length': 43, 'batch_size': 64, 'learning_rate': 0.00908746835652521, 'epochs': 15}
{'accuracy': 0.581, 'f1_pos': 0.35, 'f1_neu': 0.365, 'f1_neg': 0.723, 'eval_score': 0.396}
{'max_length': 55, 'batch_size': 128, 'learning_rate': 0.0026277353339704346, 'epochs': 6}
{'accuracy': 0.475, 'f1_pos': 0.435, 'f1_neu': 0.431, 'f1_neg': 0.535, 'eval_score': 0.443}
{'max_length': 35, 'batch_size': 128, 'learning_rate': 0.007686654539422726, 'epochs': 11}
{'a

In [ ]:
{'max_length': 31, 'batch_size': 128, 'learning_rate': 0.00043843544096825674, 'epochs': 7}
{'accuracy': 0.581, 'f1_pos': 0.474, 'f1_neu': 0.26, 'f1_neg': 0.727, 'eval_score': 0.594}

In [ ]:
# max_length = 30
# epochs = 20
# batch_size = 64
# learning_rate = 0.001
# we_dim = 200

# accuracy, f1_Positive, f1_Neutral, f1_Negative= cross_valid_cnn(obama_train_pr_df['Doc Text'],obama_train_pr_df['Sentiment'], epochs,batch_size,max_length, learning_rate, we_dim)
# f1 = round(mean([accuracy, f1_Positive, f1_Negative]),3)    
# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'f1_eval': f1}
# print('\n')
# print(eval_dict)

In [ ]:
# def objective_func_CNN_CV(args):
#     max_length = args['max_length']
#     batch_size = args['batch_size']
#     learning_rate = args['learning_rate']
#     epochs = args['epochs']

#     par_dict = {'max_length': max_length, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs}
#     print(par_dict)

#     accuracy, f1_Positive, f1_Neutral, f1_Negative= cross_valid_cnn(obama_train_pr_df['Doc Text'],obama_train_pr_df['Sentiment'], epochs,batch_size,max_length, learning_rate, we_dim)
#     f1 = round(mean([accuracy, f1_Positive, f1_Negative]),3)    
#     eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'f1_eval': f1}
#     print('\n')
#     print(eval_dict)

#     return -(f1)

# space = {'max_length': hp.choice('max_length',range(4,60)),  
#         'batch_size': hp.choice('batch_size', [32, 64, 128]),
#          'epochs': hp.choice('epochs',range(5,30)), 
#          'learning_rate': hp.uniform('learning_rate', 0,0.01)
#         }                  
                                    
# we_dim = 200                               
# best_CNN_CV = fmin(objective_func_CNN_CV, space, algo=tpe.suggest, max_evals=20)
# print(best_CNN_CV)

## Custom F1 value

In [ ]:
def f1_value(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val